<a href="https://colab.research.google.com/github/garylau1/model_training/blob/main/sea_animal_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
The diversity of marine life presents a unique opportunity to leverage deep learning for the classification and identification of sea animals. In this project, I developed a robust machine learning pipeline to classify 23 distinct species of sea animals using a pre-trained Wide ResNet-50-2 architecture. This architecture, renowned for its superior feature extraction capabilities and computational efficiency, served as the backbone of the model, allowing for accurate classification in a complex and nuanced dataset.

The model achieved an impressive accuracy exceeding 80%, demonstrating its effectiveness in discerning subtle differences among species. Furthermore, to ensure accessibility and scalability, the trained model was deployed on the Hugging Face platform, enabling seamless integration and real-world application. This report outlines the methodology, implementation, and results of the project, highlighting the significant role of pre-trained architectures in advancing marine biodiversity research.


Pre-train model link: https://pytorch.org/vision/main/models/generated/torchvision.models.wide_resnet50_2.html#torchvision.models.Wide_ResNet50_2_Weights



In [1]:
# Import necessary libraries
import torch  # Core library for PyTorch
import torchvision  # PyTorch library for vision-related tasks
import random  # Used for generating random values
import matplotlib.pyplot as plt  # For visualizing images and results

# Additional imports
from torch import nn  # Neural network modules in PyTorch
from torchvision import transforms  # For applying data transformations

# File handling and image processing
import requests  # For downloading data from the internet
import zipfile  # For extracting .zip files
from pathlib import Path  # For file path operations
from PIL import Image  # For working with image files

# Dataset handling
from torchvision import datasets  # Prebuilt datasets in torchvision
from torch.utils.data import Dataset, DataLoader  # Dataset and data loader utilities
import os  # OS module for file and directory operations

# Setting the device for computation
device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available, otherwise fallback to CPU

# 1. Import libraries for data acquisition and preparation
from pathlib import Path  # Helps in working with file paths
import requests  # Used to download files from the internet
import zipfile  # For extracting compressed files

# Mount Google Drive (specific to Google Colab)
from google.colab import drive
drive.mount('/content/drive')  # Mounts the Google Drive to access datasets and save outputs

Mounted at /content/drive


In [2]:
# Define the target path for the dataset
target_path = Path("Our_datata")  # Specify the folder to store the dataset

# Check if the target path exists
if not os.path.exists(target_path):  # If the folder doesn't exist,
    os.mkdir(target_path)  # Create the folder

import shutil  # Import shutil for file operations

# Define the source and destination paths
source_path = '/content/drive/MyDrive/sea_animal.zip'  # Path to the zip file in Google Drive

# Copy the zip file from the source path to the target folder
shutil.copy(source_path, target_path)

# Unpack (extract) the zip file into the target folder
shutil.unpack_archive("/content/Our_datata/sea_animal.zip", target_path)

# Remove the zip file after extraction to save space
os.remove("/content/Our_datata/sea_animal.zip")



# Pretrained Model:
This section defines a function to create a classification model based on the Wide ResNet-50-2 architecture.

The function modifies the pre-trained model to freeze its existing weights and replaces the final layer to adapt to


the task of classifying 23 sea animal species. The model also includes data transformations specific to the pre-trained weights.

In [19]:
def created_model(class_n=23):
    """
    Creates and customizes a Wide ResNet-50-2 model for multi-class classification.

    Args:
        class_n (int): Number of output classes for the model. Default is 23.

    Returns:
        model (torch.nn.Module): The modified Wide ResNet-50-2 model.
        transform_ (callable): Pre-defined data transformations for the model.
    """
    # Load pre-trained weights for Wide ResNet-50-2
    weights = torchvision.models.Wide_ResNet50_2_Weights.DEFAULT

    # Initialize the pre-trained Wide ResNet-50-2 model
    model = torchvision.models.wide_resnet50_2(weights)

    # Get the transformation associated with the pre-trained weights
    transform_ = weights.transforms()

    # Freeze all layers of the model to prevent updates during training
    for param in model.parameters():
        param.requires_grad = False

    # Replace the final fully connected (fc) layer with a custom classifier
    model.fc = nn.Sequential(
        nn.Dropout(p=0.4, inplace=True),  # Add dropout for regularization
        nn.Linear(model.fc.in_features, model.fc.in_features),  # Intermediate fully connected layer
        nn.ReLU(),  # Activation function
        nn.Linear(model.fc.in_features, out_features=class_n)  # Final output layer with 'class_n' outputs
    )

    # Return the customized model and the corresponding data transformation
    return model, transform_

# Create a new instance of the model and its transformation for 23 classes
new_model, transform_ = created_model(23)



# Dataset Splitting and DataLoader Setup
This section splits the dataset into training and testing subsets, with 85% of data used for training and 15% for testing. It then creates DataLoader instances for both the training and testing datasets, which will handle batching, shuffling, and parallel data loading during training and evaluation.

Finally, an Adam optimizer and a Cross-Entropy loss function with label smoothing are set up for the model.


In [93]:
# Combine additional transformations with the pre-trained model's specific transformation
final_transform = transforms.Compose([
    transforms.TrivialAugmentWide(),  # Apply a wide range of simple augmentations for data diversity
    transform_  # Include the pre-defined transformation from the pre-trained model weights
])

# Load the dataset using ImageFolder
Dataset_ = datasets.ImageFolder(
    root="/content/Our_datata",  # Path to the root directory containing the dataset
    transform=final_transform  # Apply the composed transformations to the images
)
#TypeError: Only torch.uint8 image tensors are supported, but found torch.float32




# Split the dataset into training and testing sets
# 85% for training and 15% for testing
train_dataset, test_dataset = random_split(Dataset_, [0.85, 0.15])

# Import DataLoader to load the data in batches
from torch.utils.data import DataLoader

# Create DataLoader for the training dataset
# Batch size of 16, shuffling enabled to improve training, and using 2 workers for parallel data loading
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)

# Create DataLoader for the testing dataset
# Batch size of 16, no shuffling as test data should not be randomized, and using 2 workers
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

# Initialize the Adam optimizer for the model with learning rate of 0.001
optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001)

# Define the loss function to be used during training (Cross-Entropy Loss with label smoothing)
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)

#  Model Training

This function model_training trains the model using the training data, evaluates it on the test data, and returns the average loss and accuracy for both the training and testing phases. The model, loss function, optimizer, and dataloaders are passed as arguments, and the function runs the training and evaluation loop over the dataset for one epoch.


In [7]:
def model_training(model=new_model, train_dataloader=train_dataloader, test_dataloader=test_dataloader, optimizer=optimizer, loss_fn=loss_fn, device=device):
    """
    Trains the given model on the training dataset and evaluates it on the test dataset.

    Args:
        model (torch.nn.Module): The model to be trained.
        train_dataloader (torch.utils.data.DataLoader): The DataLoader for the training dataset.
        test_dataloader (torch.utils.data.DataLoader): The DataLoader for the testing dataset.
        optimizer (torch.optim.Optimizer): The optimizer used for training.
        loss_fn (torch.nn.Module): The loss function used during training.
        device (str): The device ('cuda' or 'cpu') where the model and data should be loaded.

    Returns:
        tuple: A tuple containing the average training loss, average training accuracy, average test loss, and average test accuracy.
    """

    # Move the model to the specified device (GPU/CPU)
    model = model.to(device)

    # Set the model in training mode
    model.train()

    # Initialize variables to track the total training loss and accuracy
    train_loss = 0
    train_acc = 0

    # Initialize variables to track the total test loss and accuracy
    test_loss = 0
    test_acc = 0

    # Training loop over the training dataset
    for batch, (X, y) in enumerate(train_dataloader):
        # Move inputs and targets to the correct device
        X, y = X.to(device), y.to(device)

        # Forward pass: Compute predicted y by passing X to the model
        y_pred = model(X)

        # Compute the loss between predictions and ground truth labels
        loss = loss_fn(y_pred, y)

        # Calculate accuracy by comparing the predicted labels to the true labels
        acc = (sum(y_pred.argmax(1) == y).item()) / len(y)

        # Accumulate the loss and accuracy for training
        train_acc += acc
        train_loss += loss.item()

        # Zero the gradients before performing the backward pass
        optimizer.zero_grad()

        # Backward pass: Compute gradients of the loss w.r.t the model parameters
        loss.backward()

        # Update model parameters using the optimizer
        optimizer.step()

    # Set the model in evaluation mode (no gradient updates)
    model.eval()

    # No gradient tracking needed for inference
    with torch.inference_mode():
        # Evaluation loop over the test dataset
        for batch, (X, y) in enumerate(test_dataloader):
            # Move inputs and targets to the correct device
            X, y = X.to(device), y.to(device)

            # Forward pass: Compute predicted y by passing X to the model
            y_pred = model(X)

            # Compute the loss between predictions and ground truth labels
            loss = loss_fn(y_pred, y)

            # Calculate accuracy by comparing the predicted labels to the true labels
            acc = (sum(y_pred.argmax(1) == y).item()) / len(y)

            # Accumulate the loss and accuracy for testing
            test_acc += acc
            test_loss += loss.item()

    # Return the average loss and accuracy for both training and testing
    return train_loss / len(train_dataloader), train_acc / len(train_dataloader), test_loss / len(test_dataloader), test_acc / len(test_dataloader)



In [24]:
# We store our result:
Result={"train_loss":[],"train_acc":[],"test_loss":[],"test_acc":[]}
for i in range(30):
  train_loss,train_acc,test_loss,test_acc=model_training()
  print(f"train_loss:{train_loss:5f} |  train_acc:{train_acc:5f} |  test_loss:{test_loss:5f} | test_acc:{test_acc:5f}")
  Result["train_loss"].append(train_loss)
  Result["train_acc"].append(train_acc)
  Result["test_loss"].append(test_loss)
  Result["test_acc"].append(test_acc)


train_loss:1.058358 |  train_acc:0.859225 |  test_loss:1.165916 | test_acc:0.831880
train_loss:1.063266 |  train_acc:0.859262 |  test_loss:1.156632 | test_acc:0.832849
train_loss:1.053143 |  train_acc:0.868031 |  test_loss:1.120863 | test_acc:0.840116
train_loss:1.063431 |  train_acc:0.856837 |  test_loss:1.147990 | test_acc:0.828488
train_loss:1.047792 |  train_acc:0.866231 |  test_loss:1.170182 | test_acc:0.823643
train_loss:1.061826 |  train_acc:0.860976 |  test_loss:1.140749 | test_acc:0.833818
train_loss:1.056756 |  train_acc:0.863977 |  test_loss:1.161019 | test_acc:0.825097
train_loss:1.056866 |  train_acc:0.859801 |  test_loss:1.157367 | test_acc:0.829942
train_loss:1.034715 |  train_acc:0.872893 |  test_loss:1.142809 | test_acc:0.832364
train_loss:1.039540 |  train_acc:0.870407 |  test_loss:1.160575 | test_acc:0.832364
train_loss:1.033799 |  train_acc:0.869660 |  test_loss:1.172807 | test_acc:0.822674
train_loss:1.036714 |  train_acc:0.871044 |  test_loss:1.145317 | test_acc:0

KeyboardInterrupt: 

# Discussion of Results

The training and testing results presented above reflect the performance of the sea animal classification model, trained using the pre-trained Wide ResNet-50-2 architecture. Over the course of several epochs, the model was able to demonstrate a consistent learning pattern, with both training and testing accuracy steadily fluctuating. The following analysis will focus on the model's learning behavior, performance comparison between training and testing metrics, and possible reasons for the observed results.

Training Loss and Accuracy
From the results, we observe that the training loss fluctuates around 1.05, gradually decreasing over time. At the start of the training, the loss was 1.058, and as training progressed, it decreased slightly, indicating that the model was learning and adjusting its weights to reduce the discrepancy between predicted and actual labels. By the final epoch, the training loss had reduced further to approximately 1.0048. This shows that the model was able to minimize the error during training, which is a positive sign that the learning process was effective.

The training accuracy also follows a similar upward trend. Initially, the training accuracy was around 85.92% and gradually increased over time, with the final epoch reaching 88.5%. This suggests that the model was successfully learning to classify the sea animal species from the training dataset. However, the gradual improvement in accuracy indicates that the model may have reached a point where the performance gains were marginal, potentially pointing to the model nearing its capacity or the possibility of early convergence.

Testing Loss and Accuracy
While the model showed solid training performance, the testing results revealed slightly different dynamics. The test loss fluctuated between 1.12 and 1.17, with some higher values seen in the earlier epochs. However, in the later stages of training, the test loss stayed relatively consistent around 1.14 to 1.16, showing that the model had not significantly overfit to the training data. This consistency in test loss suggests that the model generalized well, but also indicates that further improvements in accuracy were limited.

The test accuracy ranged from 82.17% to 84.01%. While the model’s test accuracy is competitive, it consistently remained lower than the training accuracy. This difference between the training and testing performance is a typical sign of overfitting or the model not being fully generalized to unseen data. However, the fact that the test accuracy was consistently above 82% indicates that the model was able to generalize well, even though there may have been room for improvement.

# Save and Load Model
This section handles saving the trained model's state dictionary to disk and later loading it into a new model instance.
The model is saved in the "animal_model" directory, and a new instance of the model is created by loading the saved state.


In [20]:
import pathlib
# Create a directory called "animal_model" if it doesn't already exist
p = pathlib.Path("animal_model").mkdir(parents=True, exist_ok=True)

# Save the model's state_dict (weights and biases) to a file called 'animal_model_new.pth'
torch.save(new_model.state_dict(), "/content/animal_model/animal_model_new.pth")

# Create a new instance of the model using the same architecture
pre_model, transforms_ = created_model()

# Load the saved model's state_dict into the new model instance from the saved file
pre_model.load_state_dict(torch.load(f="/content/animal_model (1).pth", map_location=torch.device("cpu")))

# Print the model to verify its architecture
pre_model

<ipython-input-20-b192b80c06a2>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pre_model.load_state_dict(torch.load(f="/content/animal_model (1).pth", map_location=torch.d

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

# Deployment of out model

In this part we will store the files we needed in the folder called deployments and I will upload them into hugging face.

The trained model is deployed using the Gradio library to create a user-friendly interface for predictions.

The model is loaded into memory and set up to classify images of sea animals.

The interface allows users to upload images, get predictions with confidence scores, and measure prediction time.

In [22]:
new_dic=pathlib.Path("deployment").mkdir(parents=True,exist_ok=True)
shutil.copy("/content/animal_model (1).pth","/content/deployment")
new_example=pathlib.Path("deployment/examples").mkdir(parents=True,exist_ok=True)

shutil.copy("/content/Our_datata/Fish/101198240_431f4c276b_o.jpg","/content/deployment/examples")
shutil.copy("/content/Our_datata/Dolphin/10080987554_27d23b7ca3_o.jpg","/content/deployment/examples")
shutil.copy("/content/Our_datata/Corals/10465606544_52913a3632_o.jpg","/content/deployment/examples")


txt_path=pathlib.Path("/content/deployment")/"class_name.txt"
txt_path

with open(txt_path,"w") as f:
  f.write("\n".join(Dataset_.classes))

In [27]:
%%writefile deployment/model.py


import torch
import torchvision
from torch import nn
def created_model(class_n=23):
  weights=torchvision.models.Wide_ResNet50_2_Weights.DEFAULT


  model=torchvision.models.wide_resnet50_2(weights)
  transform_=weights.transforms()
  for i in model.parameters():
    i.requires_grad=False

  model.fc=nn.Sequential(nn.Dropout(p=0.4,inplace=True),nn.Linear(model.fc.in_features,model.fc.in_features),nn.ReLU(),nn.Linear(model.fc.in_features,out_features=class_n))
  return model,transform_


Writing deployment/model.py


In [83]:
%%writefile deployment/app.py

import torch
import torchvision
import random
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

import gradio as gr  # Gradio library for creating web interfaces
import os

from model import created_model  # Importing the model creation function
from timeit import default_timer as timer  # Timer to measure prediction time

# Create model instance and load pre-trained weights
pre_model, transforms_ = created_model()
pre_model.load_state_dict(torch.load(f="/animal_model (1).pth", map_location=torch.device("cpu")))

# Move model to CPU for inference
pre_model = pre_model.cpu()

# Define the prediction function
def prediction_fuc(img):
    """
    Function to make predictions on input images using the trained model.
    Args:
        img (PIL.Image): The input image for classification.
    Returns:
        dict: A dictionary with the class names and their respective prediction probabilities.
        float: Time taken for prediction.
    """
    start = timer()  # Start timer to measure prediction time

    # Preprocess the image (apply transformation and add batch dimension)
    img = transforms_(img).unsqueeze(0).to("cpu")

    # Set model to evaluation mode
    pre_model.eval()

    # Perform inference
    with torch.inference_mode():
        logit = pre_model(img).squeeze()  # Get raw logits (model outputs)
        pred = torch.softmax(logit, dim=-1)  # Convert logits to probabilities

    end = timer()  # End timer

    # Return predictions as class names with probabilities and prediction time
    return {Dataset_.classes[i]: round(pred[i].item(), 6) for i in range(len(Dataset_.classes))}, round(end - start, 4)

# Set up Gradio interface for user input and displaying predictions
title = "Classification of 23 Classes of Sea Animal"
description = "Wide ResNet50-2 model for classification of sea animals."

# Example images for testing the model (from the 'examples' folder)
example_list = [["examples/" + i] for i in os.listdir("examples")]

# Create Gradio interface
demo = gr.Interface(
    fn=prediction_fuc,  # The function to call for predictions
    inputs=gr.Image(type="pil"),  # Input type is an image (PIL format)
    outputs=[gr.Label(num_top_classes=4, label="Predictions"), gr.Number(label="Prediction Time (s)")],  # Outputs: top 4 predictions and time taken
    examples=example_list,  # Example images to display for users
    title=title,  # Title of the app
    description=description  # Description of the app
)

# Launch the Gradio interface
demo.launch()

Overwriting deployment/app.py


In [85]:
%%writefile deployment/requirments.txt

torch==2.5.1
torchvision==0.20.1
gradio==5.8.0

Writing deployment/requirments.txt


# Gradio Deployment for Sea Animal Classification

This section deploys the trained model using Gradio to create an interactive web interface for classifying sea animals into 23 classes. Users can upload an image, and the interface will return the top predicted classes along with the prediction time.
The model used for classification is Wide ResNet50-2, and it outputs the predicted class probabilities
and time taken to perform the classification.

In [88]:
from PIL import Image
from timeit import default_timer as timer

#we write out prediction_fuc for testing

test_image=Image.open("/content/Our_datata/Corals/10465606544_52913a3632_o.jpg")
def prediction_fuc(img):
  """
    Function to make predictions on input images using the trained model.
    Args:
        img (PIL.Image): The input image for classification.
    Returns:
        dict: A dictionary with the class names and their respective prediction probabilities.
        float: Time taken for prediction.
    """

  start=timer()
  img=transform_(img).unsqueeze(0).to("cpu")
  pre_model.eval()
  with torch.inference_mode():
      logit=pre_model(img).squeeze()
      pred=torch.softmax(logit,dim=-1)
  end=timer()
  return {Dataset_.classes[i]: round(pred[i].item(),6) for i in range(len(Dataset_.classes))},round(end-start,4)



#print out one example of prediction_function
prediction_fuc(test_image)

({'Clams': 0.003447,
  'Corals': 0.771314,
  'Crabs': 0.001092,
  'Dolphin': 0.000344,
  'Eel': 0.004419,
  'Fish': 0.159919,
  'Jelly Fish': 0.002563,
  'Lobster': 0.00181,
  'Nudibranchs': 0.000969,
  'Octopus': 0.002809,
  'Otter': 0.002182,
  'Penguin': 0.000675,
  'Puffers': 0.004549,
  'Sea Rays': 0.00503,
  'Sea Urchins': 0.004265,
  'Seahorse': 0.001014,
  'Seal': 0.000744,
  'Sharks': 0.012505,
  'Shrimp': 0.000614,
  'Squid': 0.004706,
  'Starfish': 0.002503,
  'Turtle_Tortoise': 0.004895,
  'Whale': 0.007633},
 0.5081)

In [71]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [100]:
import gradio as gr  # Gradio for creating web interfaces
import os
from timeit import default_timer as timer  # Timer to measure prediction time
from PIL import Image  # For image processing

# Function to make predictions
def prediction_fuc(img):
    """
    Function to predict the class of the input image using the trained model.

    Args:
        img (PIL.Image): The input image for classification.

    Returns:
        dict: A dictionary of class labels with their respective prediction probabilities.
        float: Time taken for making the prediction.
    """
    start = timer()  # Start timer to measure prediction time

    # Preprocess the image: apply transformations and add a batch dimension
    img = transform_(img).unsqueeze(0).to("cpu")

    # Set model to evaluation mode
    pre_model.eval()

    # Perform inference on the image with the model
    with torch.inference_mode():
        logit = pre_model(img).squeeze()  # Get raw model outputs (logits)
        pred = torch.softmax(logit, dim=-1)  # Convert logits to probabilities

    end = timer()  # End timer to measure total inference time

    # Return the class names with predicted probabilities and the time taken for prediction
    return {Dataset_.classes[i]: round(pred[i].item(), 6) for i in range(len(Dataset_.classes))}, round(end - start, 4)

# Set up the Gradio demo interface
title = "Classification of 23 Classes of Sea Animal"
description = "Wide ResNet50-2 model for classification of sea animals."

# Example images for testing the model (these images are used to demonstrate the functionality of the app)
example_list = [["/content/deployment/examples/10080987554_27d23b7ca3_o.jpg"], ["/content/deployment/examples/10465606544_52913a3632_o.jpg"]]

# Create Gradio interface
demo = gr.Interface(
    fn=prediction_fuc,  # The function to call for predictions
    inputs=gr.Image(type="pil"),  # The input is an image (PIL format)
    outputs=[gr.Label(num_top_classes=4, label="Predictions"), gr.Number(label="Prediction Time (s)")],  # Outputs: top 4 predictions and time taken
    examples=example_list,  # Provide example images for testing
    title=title,  # Set the title of the interface
    description=description  # Set the description of the interface
)

# Launch the Gradio interface and enable sharing to generate a public URL
demo.launch(debug=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e5983dd1739f69c8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
